### Package installations

In [ ]:
!pip install clarifaipyspark
!pip install protobuf==4.24.2

### Setting Environment Variables & ClarifaiPySpark Objects

In [ ]:
from clarifai.client.input import Inputs
from google.protobuf.json_format import MessageToJson
import os
import json

os.environ['CLARIFAI_PAT'] = ''

user_id = 'mansi_k'
app_id = 'databricks_tester_txt'
dataset_id = 'dataset1'

cspark_obj = ClarifaiPySpark(user_id=user_id, app_id=app_id)
dataset_obj = cspark_obj.dataset(dataset_id=dataset_id)

2023-10-26 12:39:15 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=848161;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=415770;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Fetching annotations from App to dataframe

In [ ]:
annot_df = dataset_obj.export_annotations_to_dataframe()
annot_df.show()

2023-10-26 12:43:56 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=11595;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=119292;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+----------------+--------+-------------------+-------+
|          annotation|         created_at|              id|input_id|        modified_at|user_id|
+--------------------+-------------------+----------------+--------+-------------------+-------+
|{concepts -> [{na...|10/26/% 12:43:   %5|NrOaWbbfH6QEVFBW|     t11|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|8sH9gt43eqx3rBYF|     t21|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|fc9VQUXheCJaZu28|     t31|10/26/% 12:43:   %5|mansi_k|
+--------------------+-------------------+----------------+--------+-------------------+-------+



### Creating a Delta Live Table from annotations dataframe

In [ ]:

from pyspark.sql.functions import *
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Delta Live Table Demo").getOrCreate()
database_name = "mansi_test"
table_name = "dlt_anns2"
delta_path = "/mnt/delta_anns2"
annot_df.write.format("delta").mode("overwrite").save(delta_path)

# Create a Spark session
spark.sql(f"CREATE DATABASE IF NOT EXISTS {database_name}")
spark.sql(f"USE {database_name}")
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS {table_name} 
    USING delta 
    LOCATION '{delta_path}'
""")

2023-10-26 12:44:05 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=970265;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=156233;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

DataFrame[]

### Reading the annotations Delta Live table

In [ ]:
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()

2023-10-26 12:44:16 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=453395;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=680231;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+----------------+--------+-------------------+-------+
|          annotation|         created_at|              id|input_id|        modified_at|user_id|
+--------------------+-------------------+----------------+--------+-------------------+-------+
|{concepts -> [{na...|10/26/% 12:43:   %5|8sH9gt43eqx3rBYF|     t21|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|fc9VQUXheCJaZu28|     t31|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|NrOaWbbfH6QEVFBW|     t11|10/26/% 12:43:   %5|mansi_k|
+--------------------+-------------------+----------------+--------+-------------------+-------+



### Add new inputs and annotations to the App

In [ ]:
### Done in the App directly

### Fetch new annotations from app to dataframe

In [ ]:
annot_df = dataset_obj.export_annotations_to_dataframe()
annot_df.show()

2023-10-26 12:45:56 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=46025;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=702318;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+--------------------+----------------+-------------------+-------+
|          annotation|         created_at|                  id|        input_id|        modified_at|user_id|
+--------------------+-------------------+--------------------+----------------+-------------------+-------+
|{concepts -> [{na...|10/26/% 12:45:   %5|190e2387564c416f8...|PKaXcNjJ5fJ7wZqR|10/26/% 12:45:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:45:   %5|22757a0e73fa4cac8...|IYMxwJq0jjwJguLE|10/26/% 12:45:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|    NrOaWbbfH6QEVFBW|             t11|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|    8sH9gt43eqx3rBYF|             t21|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|    fc9VQUXheCJaZu28|             t31|10/26/% 12:43:   %5|mansi_k|
+--------------------+-------------------+--------------------+----------------+-------------------+-------+



### Upsert new annotations into the Delta live table

In [ ]:
from delta.tables import DeltaTable
# Upsert the updated rows into the Delta table
delta_table = DeltaTable.forPath(spark, delta_path)
delta_table.alias("t").merge(
    annot_df.alias("s"),
    "s.id = t.id"
).whenMatchedUpdateAll() \
 .whenNotMatchedInsertAll() \
 .execute()

2023-10-26 12:46:09 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=871239;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=602808;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

### Display annotations Delta live table

In [ ]:
df_delta = spark.read.format("delta").load(delta_path)
df_delta.show()

2023-10-26 12:46:32 INFO     INFO:py4j.clientserver:Received command c on object id p0          ]8;id=409808;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py\clientserver.py]8;;\:]8;id=400430;file:///databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py#575\575]8;;\

+--------------------+-------------------+--------------------+----------------+-------------------+-------+
|          annotation|         created_at|                  id|        input_id|        modified_at|user_id|
+--------------------+-------------------+--------------------+----------------+-------------------+-------+
|{concepts -> [{na...|10/26/% 12:45:   %5|22757a0e73fa4cac8...|IYMxwJq0jjwJguLE|10/26/% 12:45:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|    8sH9gt43eqx3rBYF|             t21|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:45:   %5|190e2387564c416f8...|PKaXcNjJ5fJ7wZqR|10/26/% 12:45:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|    NrOaWbbfH6QEVFBW|             t11|10/26/% 12:43:   %5|mansi_k|
|{concepts -> [{na...|10/26/% 12:43:   %5|    fc9VQUXheCJaZu28|             t31|10/26/% 12:43:   %5|mansi_k|
+--------------------+-------------------+--------------------+----------------+-------------------+-------+

